In [ ]:
import cc3d
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.image import imread
from skimage import draw, measure
from shapely.geometry import Polygon

from mpl_toolkits.mplot3d import Axes3D
from PIL import Image
import os

## Functions

In [ ]:
from typing import Dict

def roi_matching(original_roi: Polygon, roi_to_compare: Polygon, roi_to_compare_id: int, results: Dict, plane_indicator: str) -> Dict:
    
        tmp_roi = Polygon(tmp_contours)

        iou = original_roi.intersection(roi_to_compare).area / original_roi.union(roi_to_compare).area
        proportion = original_roi.intersection(roi_to_compare).area / original_roi.area

        if original_roi.within(roi_to_compare) or roi_to_compare.within(original_roi): within = True
        else: within = False

        results[f'matching_ids_{plane_indicator}_plane'].append(roi_to_compare_id)
        results[f'full_overlap_{plane_indicator}_plane'].append(within)
        results[f'overlapping_area_{plane_indicator}_plane'].append(proportion)
        results[f'IoUs_{plane_indicator}_plane'].append(iou)
        
        return results
    

In [ ]:
def find_best_matches(results: Dict) -> Dict:
    
    for plane_indicator in ['previous', 'next']:
        if len(results[f'matching_ids_{plane_indicator}_plane']) > 0:
            max_iou = max(results[f'IoUs_{plane_indicator}_plane'])
            if max_iou >= 0.5: # Does this really make sense here? IoU could also be < 0.5 and within == False, but only because of some pixel? Max reciprocal overlap??
                index = results[f'IoUs_{plane_indicator}_plane'].index(max_iou)
            elif any(results[f'full_overlap_{plane_indicator}_plane']):
                index = results[f'full_overlap_{plane_indicator}_plane'].index(True)
            else:
                index = None
            
            if type(index) == int:
                best_matching_id = results[f'matching_ids_{plane_indicator}_plane'][index]
                iou = max_iou
                overlap = results[f'overlapping_area_{plane_indicator}_plane'][index]
            else: 
                best_matching_id, iou, overlap = None, None, None
                
            results[f'best_match_{plane_indicator}_plane'] = best_matching_id
            results[f'overlapping_area_best_match_{plane_indicator}_plane'] = overlap
            results[f'IoU_best_match_{plane_indicator}_plane'] = iou
    
    return results
            
            
            
        
        
    

In [ ]:
def trace_matches(matching_results, final_ids_assignment, current_final_id, current_plane_idx, current_plane_label_id):
    best_match_next_plane = matching_results[current_plane_idx][current_plane_label_id]['best_match_next_plane']
    next_plane_idx = current_plane_idx + 1

    if matching_results[next_plane_idx][best_match_next_plane]['final_label_id_assigned']:
        raise ValueError(f'ROI with ID {best_match_next_plane} in plane {next_plane_idx} was already assigned! :o')
    else:
        if matching_results[next_plane_idx][best_match_next_plane]['best_match_previous_plane'] != current_plane_label_id:
            raise ValueError(f'ROI with ID {best_match_next_plane} in plane {next_plane_idx} does not share best matching with previous plane!')
        else:
            matching_results[next_plane_idx][best_match_next_plane]['final_label_id_assigned'] = True
            matching_results[next_plane_idx][best_match_next_plane]['final_label_id'] = current_final_id
            final_ids_assignment[current_final_id]['plane_index'].append(next_plane_idx)
            final_ids_assignment[current_final_id]['original_label_id'].append(best_match_next_plane)

            if matching_results[next_plane_idx][best_match_next_plane]['best_match_next_plane'] != None:
                keep_tracing = True
                
            else:
                keep_tracing = False

    return matching_results, final_ids_assignment, keep_tracing

In [ ]:
def unpad_x_y_dims_in_2d_array(padded_2d_array, pad_width):
    return padded_2d_array[pad_width:padded_2d_array.shape[0]-pad_width, pad_width:padded_2d_array.shape[1]-pad_width]
    
    
def unpad_x_y_dims_in_3d_array(padded_3d_array, pad_width):
    return padded_3d_array[:, pad_width:padded_3d_array.shape[1]-pad_width, pad_width:padded_3d_array.shape[2]-pad_width]

# Create some dummy data to develop & test first ideas

In [ ]:
plane_0 = np.array([[1, 1, 1, 0, 0, 0, 0, 0, 0, 0], 
                    [1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
                    [1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

plane_1 = np.array([[1, 1, 1, 0, 0, 0, 0, 0, 0, 0], 
                    [1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
                    [1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 1, 1, 1, 0, 0, 0, 0],
                    [0, 0, 0, 1, 1, 1, 0, 0, 0, 0],
                    [0, 0, 0, 1, 1, 1, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

plane_2 = np.array([[1, 1, 1, 0, 0, 0, 0, 0, 0, 0], 
                    [1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
                    [1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 1, 1, 1, 0, 0, 0, 0],
                    [0, 0, 0, 1, 1, 1, 0, 0, 0, 0],
                    [0, 0, 0, 1, 1, 1, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
                    [0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
                    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0]])

plane_3 = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 1, 1, 1, 0, 0, 0, 0],
                    [0, 0, 0, 1, 1, 1, 0, 0, 0, 0],
                    [0, 0, 0, 1, 1, 1, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
                    [0, 1, 1, 0, 0, 0, 0, 1, 1, 1],
                    [0, 1, 1, 0, 0, 0, 0, 0, 1, 0]])

plane_4 = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
                    [0, 1, 1, 0, 0, 0, 0, 0, 0, 0]])



In [ ]:
instseg_plane_0 = np.array([[1, 1, 2, 2, 0, 0, 0, 0, 0, 0], 
                            [1, 1, 2, 2, 0, 0, 0, 0, 0, 0],
                            [1, 1, 2, 2, 0, 0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

instseg_plane_1 = np.array([[1, 1, 1, 0, 0, 0, 0, 0, 0, 0], 
                            [1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
                            [1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                            [0, 0, 0, 2, 2, 2, 0, 0, 0, 0],
                            [0, 0, 0, 2, 2, 2, 0, 0, 0, 0],
                            [0, 0, 0, 2, 2, 2, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

instseg_plane_2 = np.array([[1, 1, 1, 0, 0, 0, 0, 0, 0, 0], 
                            [1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
                            [1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                            [0, 0, 0, 2, 2, 2, 0, 0, 0, 0],
                            [0, 0, 0, 2, 2, 2, 0, 0, 0, 0],
                            [0, 0, 0, 2, 2, 2, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0, 0, 0, 0, 3, 0],
                            [0, 0, 0, 0, 0, 0, 0, 3, 3, 3],
                            [0, 0, 0, 0, 0, 0, 0, 0, 3, 0]])

instseg_plane_3 = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
                            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                            [0, 0, 0, 1, 1, 1, 0, 0, 0, 0],
                            [0, 0, 0, 1, 1, 1, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0, 0, 0, 0, 2, 0],
                            [0, 3, 3, 0, 0, 0, 0, 2, 2, 2],
                            [0, 3, 3, 0, 0, 0, 0, 0, 2, 0]])

instseg_plane_4 = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
                            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                            [0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
                            [0, 1, 1, 0, 0, 0, 0, 0, 0, 0]])



In [ ]:
instseg_plane_0 = np.array([[1, 1, 1, 0, 0, 0, 0, 0, 0, 0], 
                            [1, 1, 1, 2, 2, 0, 0, 0, 0, 0],
                            [1, 1, 2, 2, 2, 2, 0, 0, 0, 0],
                            [0, 0, 2, 2, 2, 2, 0, 0, 0, 0],
                            [0, 0, 0, 2, 2, 0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

instseg_plane_1 = np.array([[1, 1, 1, 0, 0, 0, 0, 0, 0, 0], 
                            [1, 1, 1, 2, 2, 2, 0, 0, 0, 0],
                            [1, 1, 1, 2, 2, 2, 0, 0, 0, 0],
                            [0, 0, 0, 2, 2, 2, 0, 0, 0, 0],
                            [0, 0, 0, 3, 2, 3, 0, 0, 0, 0],
                            [0, 0, 0, 3, 3, 3, 0, 0, 0, 0],
                            [0, 0, 0, 3, 3, 3, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

instseg_plane_2 = np.array([[1, 1, 1, 0, 0, 0, 0, 0, 0, 0], 
                            [1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
                            [1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                            [0, 0, 0, 2, 2, 2, 0, 0, 0, 0],
                            [0, 0, 0, 2, 2, 2, 0, 0, 0, 0],
                            [0, 0, 0, 2, 2, 2, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0, 0, 0, 0, 3, 0],
                            [0, 0, 0, 0, 0, 0, 0, 3, 3, 3],
                            [0, 0, 0, 0, 0, 0, 0, 0, 3, 0]])

instseg_plane_3 = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
                            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                            [0, 0, 0, 1, 1, 1, 0, 0, 0, 0],
                            [0, 0, 0, 1, 1, 1, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0, 0, 0, 0, 2, 0],
                            [0, 3, 3, 0, 0, 0, 0, 2, 2, 2],
                            [0, 3, 3, 0, 0, 0, 0, 0, 2, 0]])

instseg_plane_4 = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
                            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                            [0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
                            [0, 1, 1, 0, 0, 0, 0, 0, 0, 0]])



In [ ]:
zstack = np.array([plane_0, plane_1, plane_2, plane_3, plane_4])
instseg_zstack = np.array([instseg_plane_0, instseg_plane_1, instseg_plane_2, instseg_plane_3, instseg_plane_4])
labels_out_zstack, N_cc_zstack = cc3d.connected_components(zstack, return_N = True)
labels_out_instseg_zstack, N_cc_instseg_zstack = cc3d.connected_components(instseg_zstack, return_N = True)


In [ ]:
N_cc_instseg_zstack

In [ ]:
instseg_zstack

# Load actual instance segmented data

In [ ]:
import os

path = '/mnt/c/Users/dsege/TEMP/test_project2/05_instance_segmentations/'
cellpose_preds = [filename for filename in os.listdir(path) if filename.startswith('0000')]

cellpose_preds_to_stack = list()
for single_plane in cellpose_preds:
    cellpose_preds_to_stack.append(plt.imread(path + single_plane))
    
cellpose_pred_zstack = np.asarray(cellpose_preds_to_stack)

In [ ]:
pad_width = 1

padded_zstack = cellpose_pred_zstack.copy()
padded_zstack = np.pad(padded_zstack, pad_width=pad_width, mode = 'constant', constant_values = 0)
padded_zstack = padded_zstack[pad_width:padded_zstack.shape[0]-pad_width]


zstack = padded_zstack.copy()
z_dim, x_dim, y_dim = zstack.shape

results = dict()

for plane_idx in range(z_dim):
    print(f'starting with plane {plane_idx}')
    results[plane_idx] = dict()
    
    if plane_idx == 0:
        previous_plane_idx = None
        next_plane_idx = plane_idx + 1
    elif plane_idx == z_dim - 1:
        previous_plane_idx = plane_idx - 1
        next_plane_idx = None
    else:
        previous_plane_idx = plane_idx - 1
        next_plane_idx = plane_idx + 1
         
    plane = zstack[plane_idx]
    unique_label_ids = list(np.unique(plane))
    unique_label_ids.remove(0)

    for label_id in unique_label_ids:
        tmp_array = np.zeros((x_dim, y_dim), dtype='uint8')
        tmp_array[np.where(zstack[plane_idx] == label_id)] = 1
        tmp_contours = measure.find_contours(tmp_array, level = 0)[0]
        roi = Polygon(tmp_contours)
        roi_area = roi.area
        
        results[plane_idx][label_id] = {'final_label_id_assigned': False,
                                        'final_label_id': None,
                                        'area': roi_area,
                                        'matching_ids_previous_plane': list(),
                                        'full_overlap_previous_plane': list(),
                                        'overlapping_area_previous_plane': list(),
                                        'IoUs_previous_plane': list(),
                                        'matching_ids_next_plane': list(),
                                        'full_overlap_next_plane': list(),
                                        'overlapping_area_next_plane': list(),
                                        'IoUs_next_plane': list(),
                                        'best_match_previous_plane': None,
                                        'overlapping_area_best_match_previous_plane': None,
                                        'IoU_best_match_previous_plane': None,
                                        'best_match_next_plane': None,
                                        'overlapping_area_best_match_next_plane': None,
                                        'IoU_best_match_next_plane': None}
        # Reset results:
        labels_of_pixels_in_previous_plane, labels_of_pixels_in_next_plane = None, None
        
        if previous_plane_idx != None:
            labels_of_pixels_in_previous_plane = zstack[previous_plane_idx][np.where(plane == label_id)]
            labels_of_pixels_in_previous_plane = list(np.unique(labels_of_pixels_in_previous_plane))
            if 0 in labels_of_pixels_in_previous_plane:
                labels_of_pixels_in_previous_plane.remove(0)
            elif 0.0 in labels_of_pixels_in_previous_plane:
                labels_of_pixels_in_previous_plane.remove(0.0)
            
            for label_id_prev_plane in labels_of_pixels_in_previous_plane:
                tmp_array = np.zeros((x_dim, y_dim), dtype='uint8')
                tmp_array[np.where(zstack[previous_plane_idx] == label_id_prev_plane)] = 1
                tmp_contours = measure.find_contours(tmp_array, level = 0)[0]
                tmp_roi = Polygon(tmp_contours)
                
                results[plane_idx][label_id] = roi_matching(roi, tmp_roi, label_id_prev_plane, results[plane_idx][label_id], 'previous')
                
        if next_plane_idx != None:
            labels_of_pixels_in_next_plane = zstack[next_plane_idx][np.where(plane == label_id)]
            labels_of_pixels_in_next_plane = list(np.unique(labels_of_pixels_in_next_plane))
            if 0 in labels_of_pixels_in_next_plane:
                labels_of_pixels_in_next_plane.remove(0)
            elif 0.0 in labels_of_pixels_in_next_plane:
                labels_of_pixels_in_next_plane.remove(0.0)
                
            for label_id_next_plane in labels_of_pixels_in_next_plane:
                tmp_array = np.zeros((x_dim, y_dim), dtype='uint8')
                tmp_array[np.where(zstack[next_plane_idx] == label_id_next_plane)] = 1
                tmp_contours = measure.find_contours(tmp_array, level = 0)[0]
                tmp_roi = Polygon(tmp_contours)
                
                results[plane_idx][label_id] = roi_matching(roi, tmp_roi, label_id_next_plane, results[plane_idx][label_id], 'next')      

In [ ]:
for plane_id in range(z_dim):
    for label_id in results[plane_id].keys():
        results[plane_id][label_id] = find_best_matches(results[plane_id][label_id])

In [ ]:
final_ids = dict()

keep_going = True
final_label_id = 2000

for plane_idx in range(z_dim):
    for label_id in results[plane_idx].keys():
        if results[plane_idx][label_id]['final_label_id_assigned']:
            continue
        else:
            final_ids[final_label_id] = {'plane_index': list(),
                                         'original_label_id': list()}
            
            results[plane_idx][label_id]['final_label_id_assigned'] = True
            results[plane_idx][label_id]['final_label_id'] = final_label_id
            final_ids[final_label_id]['plane_index'].append(plane_idx)
            final_ids[final_label_id]['original_label_id'].append(label_id)

            # Now start tracing:
            if results[plane_idx][label_id]['best_match_next_plane'] != None:

                keep_tracing = True
                while keep_tracing:
                    results, final_ids, keep_tracing = trace_matches(results, 
                                                                     final_ids, 
                                                                     final_label_id, 
                                                                     final_ids[final_label_id]['plane_index'][-1], 
                                                                     final_ids[final_label_id]['original_label_id'][-1])


            final_label_id += 1

In [ ]:
final_ids

In [ ]:
new_zstack = zstack.copy()

In [ ]:
for final_label_id in final_ids.keys():
    for idx in range(len(final_ids[final_label_id]['plane_index'])):
        plane_index = final_ids[final_label_id]['plane_index'][idx]
        label_id = final_ids[final_label_id]['original_label_id'][idx]
        new_zstack[plane_index][np.where(new_zstack[plane_index] == label_id)] = final_label_id
        
new_zstack = unpad_x_y_dims_in_3d_array(new_zstack, pad_width)

In [ ]:
labels_out_new_zstack, N_cc_new_zstack = cc3d.connected_components(new_zstack, return_N = True)

In [ ]:
N_cc_new_zstack

In [ ]:
multi_matches_traceback = list()

for plane_idx in results.keys():
    for label_id in results[plane_idx].keys():
        condition_a = len(results[plane_idx][label_id]['matching_ids_next_plane']) > 1
        condition_b = len(results[plane_idx][label_id]['matching_ids_previous_plane']) > 1
        if condition_a or condition_b:
            multi_matches_traceback.append((plane_idx, label_id))

multi_matches_traceback

In [ ]:
results[1][7]

In [ ]:
unpadded_zstack = unpad_x_y_dims_in_3d_array(zstack, 1)
z_dim, x_dim, y_dim = unpadded_zstack.shape

In [ ]:
def load_cropped_zstack(path, file_id, minx, maxx, miny, maxy):
    filenames = [filename for filename in os.listdir(path) if filename.startswith(file_id)]
    cropped_zstack = list()
    for single_plane_filename in filenames:
        tmp_image = imread(path + single_plane_filename)
        tmp_image = tmp_image[minx:maxx, miny:maxy]
        cropped_zstack.append(tmp_image.copy())
        del tmp_image
    return np.asarray(cropped_zstack)

In [ ]:
from matplotlib.pyplot import cm

def get_color_code(label_ids):
    n_label_ids = len(label_ids)
    colormixer = cm.rainbow(np.linspace(0, 1, n_label_ids))

    color_code = dict()
    for idx in range(n_label_ids):
        color_code[label_ids[idx]] = colormixer[idx]
    
    return color_code

In [ ]:
def get_plotting_info(zstack):
    label_ids = list(np.unique(zstack))
    if 0 in label_ids:
        label_ids.remove(0)
    color_code = get_color_code(label_ids)
    
    z_dim, x_dim, y_dim = zstack.shape
    plotting_info = dict()
    for plane_index in range(z_dim):
        plotting_info[plane_index] = dict()
        
    for label_id in label_ids:
        for plane_index in final_ids[label_id]['plane_index']:
            plane = zstack[plane_index]
            if label_id in np.unique(zstack[plane_index]):
                roi = get_polygon_from_instance_segmentation(zstack[plane_index], label_id) 
                boundary_x_coords, boundary_y_coords = np.asarray(roi.boundary.xy[0]), np.asarray(roi.boundary.xy[1])
                plotting_info[plane_index][label_id] = {'color': color_code[label_id],
                                                        'boundary_x_coords': boundary_x_coords,
                                                        'boundary_y_coords': boundary_y_coords} 
    return plotting_info

In [ ]:
def get_polygon_from_instance_segmentation(single_plane: np.ndarray, label_id: int) -> Polygon:
    x_dim, y_dim = single_plane.shape
    tmp_array = np.zeros((x_dim, y_dim), dtype='uint8')
    tmp_array[np.where(single_plane == label_id)] = 1
    tmp_contours = measure.find_contours(tmp_array, level = 0)[0]
    return Polygon(tmp_contours)

In [ ]:
from typing import Tuple

def get_cropping_box_arround_centroid(roi: Polygon, half_window_size: int) -> Tuple:
    centroid_x, centroid_y = round(roi.centroid.x), round(roi.centroid.y)
    cminx, cmaxx = centroid_x - half_window_size, centroid_x + half_window_size
    cminy, cmaxy = centroid_y - half_window_size, centroid_y + half_window_size
    return cminx, cmaxx, cminy, cmaxy

In [ ]:
def plot_reconstructed_cells(preprocessed_zstack, instance_seg_zstack, final_labels_zstack, plotting_info, plane_of_interest, save=False, show=True):
    z_dim = final_labels_zstack.shape[0]
    fig = plt.figure(figsize=(15, 5*z_dim), facecolor='white')
    gs = fig.add_gridspec(z_dim, 3)
    
    for plane_index in range(z_dim):
        fig.add_subplot(gs[plane_index, 0])
        plt.imshow(preprocessed_zstack[plane_index])
        plt.ylabel(f'plane_{plane_index}', fontsize=14)
        if plane_index == 0:
            plt.title('input image', fontsize=14, pad=15)
            
    for plane_index in range(z_dim):
        fig.add_subplot(gs[plane_index, 1])
        plt.imshow(instance_seg_zstack[plane_index])
        if plane_index == 0:
            plt.title('instance segmentation', fontsize=14, pad=15)
            
    for plane_index in range(z_dim):
        fig.add_subplot(gs[plane_index, 2])
        plt.imshow(final_labels_zstack[plane_index], cmap = 'Greys_r')
        for label_id in plotting_info[plane_index].keys():
            plt.plot(plotting_info[plane_index][label_id]['boundary_y_coords'], 
                     plotting_info[plane_index][label_id]['boundary_x_coords'], 
                     c=plotting_info[plane_index][label_id]['color'], 
                     lw=3)
        if plane_index == plane_of_interest:
            plt.plot([185, 215], [200, 200], c='red', lw='3')
            plt.plot([200, 200], [185, 215], c='red', lw='3')
        if plane_index == 0:
            plt.title('connected components (color-coded)', fontsize=14, pad=15)
            
    if save:
        plt.savefig('inspected_area.png', dpi=300)
    if show:
        plt.show()
    else:
        plt.close()


In [ ]:
# the 'results' and the 'file_ids' dictionaries have to become attributes of the object
# also the corresponding z-stacks (both unpadded again): 
#   - the 'zstack' with the original instance label ids (the one we are looking for)
#   - the 'new_zstack' with the final label ids
# likewise, also the directory paths have to be accessible via the database (just as the current file_id)

def inspect_reconstructed_cells(plane_id_of_interest, label_id_of_interest, save=False, show=True):
    """
    plane_id_of_interest: is the plane_id in which the roi of interest can be found
    label_id_of_interest: is the rois` label_id in the original instance segmentation
    """
    roi = get_polygon_from_instance_segmentation(zstack[plane_id_of_interest], label_id_of_interest)
    cminx, cmaxx, cminy, cmaxy = get_cropping_box_arround_centroid(roi, 200)
    
    cropped_new_zstack = new_zstack.copy()
    cropped_new_zstack = cropped_new_zstack[:, cminx:cmaxx, cminy:cmaxy]
    
    plotting_info = get_plotting_info(cropped_new_zstack)
    
    cropped_preprocessed_zstack = load_cropped_zstack(preprocessed_path, file_id, cminx, cmaxx, cminy, cmaxy)
    cropped_instance_seg_zstack = load_cropped_zstack(instance_mask_path, file_id, cminx, cmaxx, cminy, cmaxy)
   
    plot_reconstructed_cells(cropped_preprocessed_zstack, 
                             cropped_instance_seg_zstack, 
                             cropped_new_zstack, 
                             plotting_info, 
                             plane_id_of_interest,
                             save=save,
                             show=show)
    

In [ ]:
multi_matches_traceback

In [ ]:
file_id = '0000'
preprocessed_path = '/mnt/c/Users/dsege/TEMP/test_project2/03_preprocessed_images/'
instance_mask_path = '/mnt/c/Users/dsege/TEMP/test_project2/05_instance_segmentations/'

inspect_reconstructed_cells(5, 69, save=True, show=False)